# Code Agents

When creating multi-agent systems, we have a compromize between flexibility and predictability:

* In **Agentic Workflows**, agent interactions are explicitly described, which allows us to plan different execution paths and estimate their cost / execution time.
* In **ReAct Agents**, execution path is dymanically created by an LLM. An agent takes problem description from a user, decomposes the problem into separate parts, and creates a plan, which is then revised at each step.

ReAct Agents can be created using **code generation** approach, when an LLM creates Python code to solve the problem, given the set of available tools. Framework called [smolagents](https://github.com/huggingface/smolagents) from HuggungFace exploits this approach.

In [ ]:
%pip install smolagents[mcp]

## Simple Example

Consider the simplest example, when we give an agent Internet search capability using internet search tool.

> SmolAgents contains built-in `DuckDuckGoSearchTool`, but we will create our own tool using Yandex Search API.

We will use YandexGPT through [OpenAI compatible API](https://yandex.cloud/ru/docs/foundation-models/concepts/openai-compatibility). You can try using other models as well, such as LLama or Qwen.

> **IMPORTANT**: When using YandexGPT 5 you need to pass `flatten_messages_as_text = True` parameter, because more complex requests including images are not currently supported.

Let's make sure LLM works:

In [15]:
from smolagents import OpenAIServerModel, CodeAgent, DuckDuckGoSearchTool,VisitWebpageTool
import os 

api_key = os.environ['api_key']
import os

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True
)

Now let's create search tool:

In [12]:
from smolagents import tool
import requests
import base64
import xml.etree.ElementTree as ET
import markdownify

@tool
def ysearch(text : str) -> str:
    """
    Find contents on the internet given a search string. Page titles, urls and short extract from the page are returned.
    Args:
        text: Search string to use
    """
    res = yandex_search(text)
    docs = []
    for title, url, body in res:
        #body = extract_text_from_url(url)
        if len(body)>1:
            docs.append(f"[{title}]({url})\n{body}")
    return "\n\n".join(docs)

print(ysearch("How much a meal would cost in Harbin. What should I eat, if I want to enjoy some good beer?"))
    


[Prices in Harbin 2025 prices in restaurants, prices of food and drinks...](https://hikersbay.com/prices/china/harbin?lang=en)

How much do cigarettes cost in Harbin? Cigarettes are cheaper in Harbin than in United States.
However, when you want to stay longer in Harbin - the best option is a monthly pass. Price for a monthly pass is: 13 USD (94 CNY). Answer given by: Niamh Cook - trip advisor & blogger at hikersbay. • How much does a taxi cost in Harbin, China.


[Harbin Travel Cost - Average Price of... | BudgetYourTrip.com](https://www.budgetyourtrip.com/china/harbin)

How much does it cost to travel to Harbin? (Average Daily Cost). Harbin trip costs: one week, two weeks, one month. Is Harbin expensive to visit? How much do I need for a trip to Harbin?
You should plan to spend around $64 (¥467) per day on your vacation in Harbin. This is the average daily price based on the expenses of other visitors. Past travelers have spent, on average for one day


[Harbin Food | Harbin Cuisine,

Теперь создадим агента с инструментом поиска в интернет, и зададим ему какой-нибудь вопрос:

In [17]:
agent = CodeAgent(tools=[ysearch,VisitWebpageTool()], model=ygpt_model)

agent.run("How much a meal would cost in Harbin, if I want to enjoy something with good beer? Give specific food items to take and their cost")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How much a meal would cost in Harbin, if I want to enjoy something with good beer? Give specific food items to  │
│ take and their cost                                                                                             │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  search_results = ysearch("cost of meals with good beer in Harbin")                                               
  print(search_results)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[Harbin Travel Cost - Average Price of... | BudgetYourTrip.com](https://www.budgetyourtrip.com/china/harbin)

While restaurant and meal prices in Harbin can vary significantly, the average cost of food in Harbin is $24 (¥174)
per day, per person, based on the spending habits of previous travelers. Food2 Meals for one day.
Harbin is considered to be one of the less traditional cities in China, meaning they have a pretty lively night 
life! You'll find a lot of people start the night in cheap Hot Pot restaurants drinking local beers, then move on 
to some of the dancing clubs. No matter what your preferred night out involves, you're bound to find it in Harbin! 
Food & Dining Tips.


[Cost of Living in Harbin. Feb 2025. Prices in Harbin](https://www.numbeo.com/cost-of-living/in/Harbin-China)

Meal for 2 People, Mid-range Restaurant, Three-course.
Prices in Harbin. Every price you add makes our comparisons more accurate and useful! Do you live in Russia?


[Prices in Harbin 2025 prices in restaurants, prices of food and 
drinks...](https://hikersbay.com/prices/china/harbin?lang=en)

In Harbin, a typical fast food meal costs: 5.60 USD (40 CNY) for a McMeal at McDonalds or BurgerKing (or similar 
combo meal), and 1.70 USD (12 CNY) for a cheeseburger. For coffee lovers: cappuccino coffee 2.70 USD (20 CNY) and 
espresso 1.90 USD (14 CNY). More information about meals budget you can find in our: Travel cost guide.


[Harbin: Cost of Living, Salaries, Prices for Rent & food](https://livingcost.org/cost/china/harbin)

The average cost of living in Harbin is $659, which is in the top 20% of the least expensive cities in the world, 
ranked 7395th out of 9294 in our global list and 90th out of 544 in China. The median after-tax salary is $629, 
which is enough to cover living expenses for 1 months.
Prices for goods and services in Harbin are partly crowdsourced by our visitors, just like yourself. Please 
consider telling us about your city.


[Eating and Drinking Cost in China, China Food 
Prices](https://www.chinahighlights.com/travelguide/chinese-food/eating-drinking-cost.htm)

What It Costs to Eat and Drink in China - Breakfast, Lunch, Dinner, Snacks, and Drinks. The average cost of food in
China is quite cheap. But it still depends on which city and restaurant you choose. Read on to get a general idea 
of food and drink prices in China to help you plan your travel budget.
The cost for drinks in China is at least half the price in the West. The cheapest mineral water is about 2 yuan for
a 500 ml bottle, while a similar bottle of cola is around4yuan. A can/bottle of beer is usually below 10 yuan. You 
can also see many milk tea / fruit juice shops in cities.


[THE 10 BEST Restaurants in Harbin (Updated July 2025) - 
Tripadvisor](https://www.tripadvisor.com/Restaurants-g297433-Harbin_Heilongjiang.html)

Breakfast Restaurants in Harbin Lunch Restaurants in Harbin Dinner Restaurants in Harbin Food Delivery Restaurants 
in Harbin Restaurants for Special Occasions in Harbin Romantic Restaurants in Harbin Cheap Eats in Harbin Late 
Night Restaurants in Harbin Restaurants with Outdoor Seating in Harbin Private Dining Restaurants in Harbin Coffee 
& Tea in Harbin Desserts in Harbin Bakeries in Harbin.


[Harbin Food | Harbin Cuisine, Street Food, Snacks & Restaurant 
Guide](https://www.chinadiscovery.com/heilongjiang/harbin/harbin-food.html)

Our Harbin food guide shows you famous and typical Harbin food, like dumplings, sausage, ice cream, street food, 
etc.
The beer has a distinctive refreshing European flavor and tastes better when it is cold. If you are in Harbin, you 
will find people chatting and enjoying the beer at barbecue stands, bars or restaurants, even in the cold winter 
days.


[Harbin Dinning: Local Snacks, Western and Vegetarian 
Food](https://www.travelchinaguide.com/cityguides/heilongjiang/harbin/when-to-go.htm)

Harbin Dining -- Where to Eat. Local Snacks. Lie Ba Lie Ba is a kind of large Russian bread and ca

[Step 1: Duration 3.16 seconds| Input tokens: 2,387 | Output tokens: 62]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  print("A basic meal/lunch can cost between $4.72 to $7.")                                                        
  print("A local beer costs around $2.")                                                                           
  print("A meal for two in a neighborhood pub can cost around $13.")                                               
  final_answer("A basic meal/lunch can cost between $4.72 to $7, and a local beer costs around $2. A meal for two  
  in a neighborhood pub can cost around $13.")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
A basic meal/lunch can cost between $4.72 to $7.
A local beer costs around $2.
A meal for two in a neighborhood pub can cost around $13.

Out - Final answer: A basic meal/lunch can cost between $4.72 to $7, and a local beer costs around $2. A meal for 
two in a neighborhood pub can cost around $13.

[Step 2: Duration 3.40 seconds| Input tokens: 6,235 | Output tokens: 222]

'A basic meal/lunch can cost between $4.72 to $7, and a local beer costs around $2. A meal for two in a neighborhood pub can cost around $13.'

## Model Context Protocol

In the example above, we have searched the internet. However, if we want an agent to find food in specific restaurant, a good choice would be to use [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction). It is a standard way to implement "remote RAG and tool calling".

Let's support our restaurant has some [food menu](../data/menu/food_en.md) and [drinks menu](../data/menu/drinks_en.md). We have implemented [simple MCP-server](../mcp-server/mcp-rest.py), which offers two tools:
* `get_food_menu` to get food menu as  markdown table
* `get_drinks_menu` to get drinks menu

To run the server, we need [FastMCP](https://github.com/jlowin/fastmcp) installed, and then run in the command line:
```bash
fastmcp run -p 8000 --host 127.0.0.1 -t sse mcp-rest-en.py
```

In [21]:
from smolagents import OpenAIServerModel, CodeAgent
import os
import yaml

ygpt_model = OpenAIServerModel(api_key=os.environ['api_key'], 
                           model_id=f"gpt://{os.environ['folder_id']}/yandexgpt/rc",
                           #model_id=f"gpt://{os.environ['folder_id']}/llama/rc",
                           api_base="https://llm.api.cloud.yandex.net/v1",
                           flatten_messages_as_text = True)

We can now use MCP tools in our agent:

In [22]:
import yaml
from smolagents.mcp_client import MCPClient

with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as tools:
    agent = CodeAgent(tools=tools, model=ygpt_model)
    agent.run("Which is the most expesive dish?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Which is the most expesive dish?                                                                                │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  food_menu = get_food_menu()                                                                                      
  print(food_menu)                                                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
| Dish Name                     | Description                                                                 | 
Type     | Price (RUB) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|-------------|  
| **"Bull on Edge" Steak**      | Juicy ribeye with a pink center, slow-cooked in the smoke of Argentine passion. 
Served with Himalayan gold salt. | Steak    | 2500        |  
| **"Tenderness of an Enraged Cow" Steak** | Filet mignon marinated in pomegranate juice and the chef's whispers. 
Plus "Dragon Wing-Style" potatoes. | Steak    | 1800        |  
| **"Steak for a Modest Oligarch"** | Beef cheek braised for 12 hours in wine from Napoleon's cellar (well, 
almost). With crispy zucchini chips. | Steak    | 1200        |  
| **"Salmon Dreaming of Norway"** | Delicate fillet baked under a crust of "mysterious northern herbs" (dill). 
Served with lemon breeze. | Fish     | 1900        |  
| **"Octopus That Saw Too Much"** | Grilled squid with ink sauce and a hint of Mediterranean secrets. For bold 
gourmets. | Fish     | 2200        |  
| **"Dorado in a Fur Coat"**   | Dorado fillet wrapped in a crispy potato shell. With "Atlantis Mystery" sauce. | 
Fish     | 1600        |  
| **"Potato 'I Can Be Truffle Too'"** | Tender slices roasted with rosemary and a hint of luxury. Served with 
Pyrenean air. | Side     | 600         |  
| **"Mushrooms That Danced Under the Moon"** | Grilled champignons with garlic sauce and a psychedelic twist. | 
Side     | 450         |  
| **"Croissants, But Not Those"** | Honey-glazed mini-carrots evoking a Parisian childhood. | Side     | 350       
|  
| **"Chocolate Apocalypse"**   | 72% cocoa dessert that detonates taste buds. With "Heavenly Cloud" ice cream. | 
Dessert  | 800         |  
| **"Cheesecake That Saw God"** | Fluffy, with caramel made from angel tears and "paradise cookie" crumbs. | 
Dessert  | 750         |  
| **"Tiramisu for Adults"**    | Classic, but with a "secret ingredient" (coffee—what did *you* think?). | Dessert 
| 700         |  

Out: None

[Step 1: Duration 1.64 seconds| Input tokens: 2,323 | Output tokens: 71]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("'Bull on Edge' Steak")                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: 'Bull on Edge' Steak

[Step 2: Duration 1.21 seconds| Input tokens: 5,394 | Output tokens: 135]

## Creating the Dinner Course

We will now create an agent to propose dinner course for us, using food-wine matching [based on the food-wine matching table](../data/food_wine_table_en.md):

In [26]:
import pandas as pd
import re 

foodmatch = [ x.strip() for x in open("../data/food_wine_table_en.md", encoding="utf8").readlines()[2:-1]]
foodmatch = [ { "food" : x.split('|')[1].strip(), "wine" : x.split('|')[2].strip()} for x in foodmatch]
df = pd.DataFrame(foodmatch)
df

,food,wine
0,Eggplant baked with cheese,Red wine: Medium-bodied* dry wines—Grenache (G...
1,Delicate lamb (lamb fillet or rack),"Red wine: Aged dry wines from Pinot Noir, Menc..."
2,"Spicy lamb: grilled, roasted, or stewed with s...","Red wine: Dry wines from Cabernet Sauvignon, R..."
3,Beef Stroganoff,"White wine: Oak-aged Chardonnay, Pinot Grigio ..."
4,Pancakes with beef filling,"Strong drinks: Vodka, Polugar, Khrenovukha. Al..."
...,...,...
131,Spiced chicken skewers,Rosé wine: Full-bodied and bold from France (T...
132,Spicy pork skewers (vinegar-onion marinade),Red wine: Dry and off-dry Sangiovese (Chianti)...
133,Chocolate,"Tea: Black, herbal, berry-fruit. Do not offend..."
134,Classic éclair,"White wine: Sweeter than the dessert—aged ""cre..."


We will explicitly create food and wine matching functions that will be based on semantic embeddings:

In [24]:
from yandex_cloud_ml_sdk import YCloudML

sdk = YCloudML(folder_id=os.environ['folder_id'], auth=os.environ['api_key'])
query_model = sdk.models.text_embeddings('query')
doc_model = sdk.models.text_embeddings('doc')

Let's compute embeddings for all lines in the table.

> **WARNING**: As this will take time, I propose to skip this call during demo, and load pre-processed table.

In [27]:
df['food_embed'] = df['food'].apply(lambda x : doc_model.run(x))
df['wine_embed'] = df['wine'].apply(lambda x : doc_model.run(x))
df.to_pickle("../data/food_wine_table_en.pkl")

In [30]:
df = pd.read_pickle("../data/food_wine_table_en.pkl")

Now we will define `find_matching_food` function tool:

In [32]:
from scipy.spatial.distance import cdist
import numpy as np
from smolagents import tool

@tool
def find_matching_food(wine : str) -> str:
    """
    Find a dish corresponding to a given wine
    Args:
        wine: Description of a wine, for which to find matching dish
    """
    wine_embed = query_model.run(wine)
    we = np.array([np.array(x) for x in df['wine_embed']])
    dist = cdist([wine_embed], we, metric='cosine')[0]
    return "Here are the best matching dishes:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['food']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_food("dry red wine"))


Here are the best matching dishes:
1. Spicy lamb: grilled, roasted, or stewed with spices
2. Steamed/poached red fish (salmon, trout)
3. Meat ravioli


The same function for finding wine:

In [34]:
@tool
def find_matching_wine(food : str) -> str:
    """
    Find matching wine for a given dish
    Args:
        food: Description of a dish, for which to find matching wine 
    """
    food_embed = query_model.run(food)
    fe = np.array([np.array(x) for x in df['food_embed']])
    dist = cdist([food_embed], fe, metric='cosine')[0]
    return "Best matching wines:\n" + "\n".join(
        [f"{j+1}. {df.iloc[i]['wine']}" 
        for j, i in enumerate(dist.argsort()[0:3])])

print(find_matching_wine('Salmon steak'))

Best matching wines:
1. White wine: Dry Cortese (Gavi), Arneis, Vermentino, Albariño, slightly off-dry Riesling, aged Pinot Grigio.
2. Red wine: For rare—aged "noble" Tempranillo (Ribera del Duero Reserva+), Sangiovese (Chianti Riserva, Brunello), "Super Tuscan," Right Bank Bordeaux, silky Argentine Malbec. For medium/well-done—dry/off-dry Syrah (Shiraz), Cabernet Sauvignon, full-bodied Malbec, Primitivo, Zinfandel, aged Aglianico, aged Rhône blends (Grenache + Syrah + Mourvèdre), Priorat (6–8+ years aging).
3. Red wine: Dry and off-dry Garnacha (Grenache), Merlot, Carmenère, Mencía, full-bodied Pinot Noir (worldwide), Russian Krasnostop, Gamay (Beaujolais-Villages).


Let's now put it all together in a single ReAct Agent:

In [37]:
with MCPClient({"url" : "http://127.0.0.1:8000/sse"}) as res_tools:
    agent = CodeAgent(
        tools=res_tools+[find_matching_food,find_matching_wine], 
        model=ygpt_model)
    agent.run("""
    Prepare a menu for a dinner which consists of a main course, matching wine and a dessert.
    Print the markdown table of food/drink items to order with their names and price. Calculate the total price.
    I want steak!
    """)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Prepare a menu for a dinner which consists of a main course, matching wine and a dessert.                       │
│     Print the markdown table of food/drink items to order with their names and price. Calculate the total       │
│ price.                                                                                                          │
│     I want steak!                                                                                               │
│                                                                                                                 │
╰─ OpenAIServerModel - gpt://b1gbicod0scglhd49qs0/yandexgpt/rc ───────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  food_menu = get_food_menu()                                                                                      
  print(food_menu)                                                                                                 
  drinks_menu = get_drinks_menu()                                                                                  
  print(drinks_menu)                                                                                               
  matching_wine = find_matching_wine(food="steak")                                                                 
  print(f"Matching wine for steak: {matching_wine}")                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
| Dish Name                     | Description                                                                 | 
Type     | Price (RUB) |  
|-------------------------------|-----------------------------------------------------------------------------|----
------|-------------|  
| **"Bull on Edge" Steak**      | Juicy ribeye with a pink center, slow-cooked in the smoke of Argentine passion. 
Served with Himalayan gold salt. | Steak    | 2500        |  
| **"Tenderness of an Enraged Cow" Steak** | Filet mignon marinated in pomegranate juice and the chef's whispers. 
Plus "Dragon Wing-Style" potatoes. | Steak    | 1800        |  
| **"Steak for a Modest Oligarch"** | Beef cheek braised for 12 hours in wine from Napoleon's cellar (well, 
almost). With crispy zucchini chips. | Steak    | 1200        |  
| **"Salmon Dreaming of Norway"** | Delicate fillet baked under a crust of "mysterious northern herbs" (dill). 
Served with lemon breeze. | Fish     | 1900        |  
| **"Octopus That Saw Too Much"** | Grilled squid with ink sauce and a hint of Mediterranean secrets. For bold 
gourmets. | Fish     | 2200        |  
| **"Dorado in a Fur Coat"**   | Dorado fillet wrapped in a crispy potato shell. With "Atlantis Mystery" sauce. | 
Fish     | 1600        |  
| **"Potato 'I Can Be Truffle Too'"** | Tender slices roasted with rosemary and a hint of luxury. Served with 
Pyrenean air. | Side     | 600         |  
| **"Mushrooms That Danced Under the Moon"** | Grilled champignons with garlic sauce and a psychedelic twist. | 
Side     | 450         |  
| **"Croissants, But Not Those"** | Honey-glazed mini-carrots evoking a Parisian childhood. | Side     | 350       
|  
| **"Chocolate Apocalypse"**   | 72% cocoa dessert that detonates taste buds. With "Heavenly Cloud" ice cream. | 
Dessert  | 800         |  
| **"Cheesecake That Saw God"** | Fluffy, with caramel made from angel tears and "paradise cookie" crumbs. | 
Dessert  | 750         |  
| **"Tiramisu for Adults"**    | Classic, but with a "secret ingredient" (coffee—what did *you* think?). | Dessert 
| 700         |  
| Wine / Drink               | Winery / Producer               | Year | Type            | Price per glass (RUB) |  
|----------------------------|----------------------------------|------|-----------------|----------------------|  
| Cabernet Sauvignon         | Château Lafite Rothschild, France | 2018 | Red Wine        | 4500                 | 
| Merlot                     | Marchesi Antinori, Italy         | 2019 | Red Wine        | 2800                 |  
| Pinot Noir                 | Domaine de la Romanée-Conti, France | 2017 | Red Wine        | 5200                 
|  
| Shiraz                     | Penfolds, Australia              | 2018 | Red Wine        | 3200                 |  
| Malbec                     | Catena Zapata, Argentina         | 2020 | Red Wine        | 2900                 |  
| Sauvignon Blanc            | Cloudy Bay, New Zealand          | 2021 | White Wine      | 3800                 |  
| Chardonnay                 | Louis Latour, France             | 2020 | White Wine      | 3500                 |  
| Riesling                   | Dr. Loosen, Germany              | 2019 | White Wine      | 3100                 |  
| Pinot Grigio               | Santa Margherita, Italy          | 2021 | White Wine      | 2700                 |  
| Gewürztraminer             | Trimbach, France                 | 2020 | White Wine      | 3300                 |  
| Scotch Whisky              | Macallan 12 Years, Scotland      | –    | Strong Alcohol  | 2500                 |  
| Tequila                    | Don Julio Añejo, Mexico          | –    | Strong Alcohol  | 2200                 |  
| Mojito                     | House Cocktail                   | –    | Cocktail        | 600                  |  
| Bloody Mary                | House Cocktail                   | –    | Cocktail        | 650                  |  
| Orange Juice             

[Step 1: Duration 3.57 seconds| Input tokens: 2,466 | Output tokens: 118]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  steak_price = 2500                                                                                               
  wine_price = 4500                                                                                                
  dessert_price = 800                                                                                              
  total_price = steak_price + wine_price + dessert_price                                                           
  final_answer(f"Menu:\nSteak: 'Bull on Edge' - {steak_price} RUB\nWine: Cabernet Sauvignon - {wine_price}         
  RUB\nDessert: 'Chocolate Apocalypse' - {dessert_price} RUB\nTotal price: {total_price} RUB")                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Menu:
Steak: 'Bull on Edge' - 2500 RUB
Wine: Cabernet Sauvignon - 4500 RUB
Dessert: 'Chocolate Apocalypse' - 800 RUB
Total price: 7800 RUB

[Step 2: Duration 3.60 seconds| Input tokens: 6,635 | Output tokens: 293]

## Takeaway

Code Agents is a way to describe very flexible agents that can plan steps to solve a problem and can execute it. However, it is difficult to predict which steps would be taken each time, and whether the agent will succeed. Coding agents are a good architecture for research problems (similar to OpenAI Deep Research).